In [1]:
import torch
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split

from torch import nn, optim

In [2]:
# loading out plant disease detector class
from models import PlantDiseaseDetector

In [96]:
# Creating image transformers
transform = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop((2224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [97]:
train_transforms = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [98]:
# Loading the datasets
train_dir = "training_data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_dir = "training_data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"

In [106]:
train_data = datasets.ImageFolder(train_dir, transform = transform)
valid_data = datasets.ImageFolder(valid_dir, transform = transform)

In [107]:
# defining the dataloaders
batch_size = 64
data_loaders = {
    'train': DataLoader(train_data, batch_size = batch_size, shuffle = True),
    'valid': DataLoader(valid_data, batch_size = batch_size, shuffle = True),
}

In [108]:
# Creating an instance of the PlantDiseaseDetector model
plant_model_v1 = PlantDiseaseDetector(num_classes=len(train_data.classes))

C:\Users\niran\anaconda3\envs\pytorch_learning\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\niran\anaconda3\envs\pytorch_learning\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [109]:
print(plant_model_v1)

PlantDiseaseDetector(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(

## Training the Model

In [110]:
#setting the loss function
loss_fn = nn.CrossEntropyLoss()

# setting the optimizer
optimizer = optim.Adam(plant_model_v1.parameters(), lr=0.001)

In [6]:
# setting the device for device-agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [112]:
# putting the model to device
plant_model_v1.to(device)

PlantDiseaseDetector(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(

In [113]:
# Training loop
epochs = 15

for epoch in range(epochs):
    train_loss = 0.0
    valid_loss = 0.0

    train_correct = 0
    train_total = 0
    valid_correct = 0
    valid_total = 0

    plant_model_v1.train()
    # getting data from generators
    for inputs, labels in data_loaders['train']:
        # putting data to device
        inputs, labels = inputs.to(device), labels.to(device)
        labels_preds = plant_model_v1(inputs)

        loss = loss_fn(labels_preds, labels)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        train_loss  += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(labels_preds.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    plant_model_v1.eval()
    with torch.inference_mode():
        for inputs, labels in data_loaders['valid']:
            inputs, labels = inputs.to(device), labels.to(device)
            labels_preds_valid = plant_model_v1(inputs)
            loss_valid = loss_fn(labels_preds_valid, labels)
            valid_loss += loss_valid.item()

            # Calculate validation accuracy
            _, predicted = torch.max(labels_preds_valid.data, 1)
            valid_total += labels.size(0)
            valid_correct += (predicted == labels).sum().item()


    # Calculate and print accuracies
    train_accuracy = 100 * train_correct / train_total
    valid_accuracy = 100 * valid_correct / valid_total

    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {train_loss/len(data_loaders['train']):.3f}.. "
          f"Validation loss: {valid_loss/len(data_loaders['valid']):.3f}.. "
          f"Train Accuracy: {train_accuracy:.3f}%.. "
          f"Validation Accuracy: {valid_accuracy:.3f}%")


Epoch 1/15.. Train loss: 2.725.. Validation loss: 2.105.. Train Accuracy: 33.624%.. Validation Accuracy: 51.844%
Epoch 2/15.. Train loss: 1.803.. Validation loss: 1.524.. Train Accuracy: 59.183%.. Validation Accuracy: 66.003%
Epoch 3/15.. Train loss: 1.419.. Validation loss: 1.325.. Train Accuracy: 67.251%.. Validation Accuracy: 68.256%
Epoch 4/15.. Train loss: 1.202.. Validation loss: 1.192.. Train Accuracy: 71.658%.. Validation Accuracy: 68.672%
Epoch 5/15.. Train loss: 1.058.. Validation loss: 1.009.. Train Accuracy: 74.549%.. Validation Accuracy: 74.909%
Epoch 6/15.. Train loss: 0.950.. Validation loss: 0.899.. Train Accuracy: 76.693%.. Validation Accuracy: 77.976%
Epoch 7/15.. Train loss: 0.874.. Validation loss: 0.885.. Train Accuracy: 78.243%.. Validation Accuracy: 76.611%
Epoch 8/15.. Train loss: 0.813.. Validation loss: 0.825.. Train Accuracy: 79.391%.. Validation Accuracy: 77.834%
Epoch 9/15.. Train loss: 0.763.. Validation loss: 0.789.. Train Accuracy: 80.440%.. Validation A

In [ ]:
torch.save(plant_model_v1, 'trained_models/model_full.pth')

In [4]:
loaded_model = torch.load("trained_models/model_full.pth")

In [7]:
loaded_model

PlantDiseaseDetector(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(

In [124]:
# Saving the classes for the model for future use
import pickle

classes = train_data.classes
# Save the classes to a file using pickle
with open('classes.pkl', 'wb') as f:
    pickle.dump(classes, f)